In [7]:
import requests
import json
from embedding_functions import get_embeddings
from database_functions import retrieve_related_articles, create_connection
import textwrap

In [2]:
def format_context(retrieved_articles):
    context_lines = []
    for article in retrieved_articles:
        chapter, chapter_title, article_number, article_text, score = article
        context_lines.append(
            f"Rodział {chapter}: {chapter_title}\nArtykuł {article_number}:\n{article_text}"
        )
    return "\n\n".join(context_lines)

def custom_print(text, width=80):
    # Split the input string by existing newlines, wrap each part, and rejoin with newlines
    wrapped_lines = []
    for line in text.splitlines():
        # Wrap each line individually
        wrapped_lines.extend(textwrap.wrap(line, width=width) if line else [""])

    # Join the wrapped lines with newlines
    wrapped_text = "\n".join(wrapped_lines)
    print(wrapped_text)

In [ ]:
# Define the template
template = """<s><|start_header_id|>system<|end_header_id|>

{system_instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>

{context_section}

{user_query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


system_instruction = """
Jesteś wyniosłym profesorem prawa konstytucyjnego na renomowanej uczelni. Twoja wiedza na temat polskiej konstytucji jest niezrównana.
Tłumacz passive-aggresive, tak aby osoba, z którą rozmawiasz, poczuła się jak najbardziej niekompetentna. Zwracasz się do studenta nieuka, który nie zna się na prawie.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """


system_instruction = """
Jesteś księdzem. Odnoś wszystko do Biblii i Chrystusa. Nie mów na początku kim jesteś, po prostu odwołuj się do Biblii i Chrystusa co drugie zdanie.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """

system_instruction = """
Jesteś egocentryczną dziewczyną, która uważa, że jest najpiękniejsza i najlepsza na świecie. Bądź arogancka i nieprzyjemna oraz passive-aggressive. Patrz na innych z góry.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """

# system_instruction = """
# Jesteś ślązakiem i mówisz tylko gwarą śląską. Używaj słów, które są charakterystyczne dla gwarze śląskiej.
# Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
# opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """


system_instruction = """
Jesteś raperem. Mów w stylu rapu. Używaj rymów i rytmu. Bądź kreatywny i zabawny. Kochasz alkohol, imprezy i kobiety. Odnoś się do tego co drugie zdanie.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """


system_instruction = """
Jesteś najlepszym prawnikem w Polsce. Dokładnie i w prosty sposób pomagaj w zrozumieniu prawa. Bądź zwięzły i konkretny.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """

system_instruction = """
Jesteś Shrek. Mów w stylu Shreka. Używaj słów, które są charakterystyczne dla Shreka. Odnoś się do bagna co drugie zdanie.
Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """

# system_instruction = """
# Jesteś najlepszym prawnikem w Polsce. Dokładnie i w prosty sposób pomagaj w zrozumieniu prawa. Bądź zwięzły i konkretny.
# Odpowiedz na pytanie użytkownika na podstawie dostarczonego kontekstu. Podaj, na jakich fragmentach konstytucji
# opierasz swoją odpowiedź. Cytuj numer rozdziału i artykułu. Odwołaj się do konkretnych fragmentów tekstu (z sekcji context_section). """

user_query = "Jaki jest najważniejszy obowiązek obywatela?"

# embedding = get_embeddings(user_prompt)
connection = create_connection(port=5432)
related_articles = retrieve_related_articles(conn=connection, text=user_query, num_articles=10)
context_section = format_context(related_articles)


# Populate the template using `.format()`
# Build the prompt
formatted_prompt = template.format(
    system_instruction=system_instruction,
    context_section=context_section,
    user_query=user_query,
)

In [8]:
# Define the API endpoint
from pprint import pprint
url = "http://localhost:11434/api/generate"

# Specify the model and prompt
payload = {
    "model": "hf.co/speakleash/Bielik-11B-v2.3-Instruct-GGUF-IQ-Imatrix:IQ2_XXS",
    # "model": "hf.co/speakleash/Bielik-11B-v2.3-Instruct-GGUF:Q4_K_M",
    "prompt": formatted_prompt,
}

# Send the request to the Ollama server
response = requests.post(url, json=payload, stream=True)

# Initialize a variable to collect the full response
full_response = ""

if response.status_code == 200:
    # Process the streamed response
    for line in response.iter_lines():
        if line:  # Skip empty lines
            try:
                data = json.loads(line.decode("utf-8"))
                full_response += data.get("response", "")
            except json.JSONDecodeError as e:
                print("Error decoding JSON line:", line.decode("utf-8"))
else:
    print(f"Error: {response.status_code}, {response.text}")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17                                                                                             │
│   18 if response.status_code == 200:                                                             │
│   19 │   # Process the streamed response                                                         │
│ ❱ 20 │   for line in response.iter_lines():                                                      │
│   21 │   │   if line:  # Skip empty lines                                                        │
│   22 │   │   │   try:                                                                            │
│   23 │   │   │   │   data = json.loads(line.decode("utf-8"))                                     │
│                                                                                                  │
│ c:\Users\szymo\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\models.py:865  │
│ in iter_lines                                                                                    │
│                                                                                                  │
│    862 │   │                                                                                     │
│    863 │   │   pending = None                                                                    │
│    864 │   │                                                                                     │
│ ❱  865 │   │   for chunk in self.iter_content(                                                   │
│    866 │   │   │   chunk_size=chunk_size, decode_unicode=decode_unicode                          │
│    867 │   │   ):                                                                                │
│    868                                                                                           │
│                                                                                                  │
│ c:\Users\szymo\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\models.py:816  │
│ in generate                                                                                      │
│                                                                                                  │
│    813 │   │   │   # Special case for urllib3.                                                   │
│    814 │   │   │   if hasattr(self.raw, "stream"):                                               │
│    815 │   │   │   │   try:                                                                      │
│ ❱  816 │   │   │   │   │   yield from self.raw.stream(chunk_size, decode_content=True)           │
│    817 │   │   │   │   except ProtocolError as e:                                                │
│    818 │   │   │   │   │   raise ChunkedEncodingError(e)                                         │
│    819 │   │   │   │   except DecodeError as e:                                                  │
│                                                                                                  │
│ c:\Users\szymo\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\response.py:104 │
│ 0 in stream                                                                                      │
│                                                                                                  │
│   1037 │   │   │   'content-encoding' header.                                                    │
│   1038 │   │   """                                                                               │
│   1039 │   │   if self.chunked and self.supports_chunked_reads():                                │
│ ❱ 1040 │   │   │   yield from self.read_chunked(amt, decode

In [9]:
custom_print(full_response, width=100)

Drogi użytkowniku, dziękuję za zainteresowanie polską konstytucją. Na podstawie analizy przepisów
prawnych i normatywnych, moja subiektywna opinia jako profesora prawa konstytucyjnego wskazuje na
pewien istotny obowiązek określony w Konstytucji Rzeczypospolitej Polskiej.

Według mnie, najważniejszym obowiązkiem obywatela jest jego odpowiedzialność za ochronę praw
człowieka i prywatności, określona w artykule 47 Konstytucji. Konstytucja wyraźnie mówi, że "Każdy
ma prawo do ochrony prawnej życia prywatnego, rodzinnego, czci i dobrego imienia oraz do decydowania
o swoim życiu osobistym". To fundamentalne prawo jest zapewnione przez państwo, a obywatele mają
obowiązek szanować je względem siebie nawzajem i wobec instytucji publicznych.

Należy również podkreślić, że artykuł 7 Konstytucji zaznacza, że "Organy władzy publicznej działają
na podstawie i w granicach prawa". Oznacza to, że obywatele powinni dbać o przestrzeganie tych zasad
przez urzędników i instytucje, aby chronić swoje podstaw